# MLM Training With Trainer

In this notebook we'll cover the training process for masked-language modeling (MLM) using the HuggingFace `Trainer` function.

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We'll be using *Meditations* by *Marcus Aurelius* as our training data. The file below has already been cleaned and so no further processing is required (beyond the `split`).

In [3]:
with open('../../data/text/meditations/clean.txt', 'r') as fp:
    text = fp.read().split('\n')

In [4]:
text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

First, we'll tokenize our text.

In [5]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

Then we create our *labels* tensor by cloning the *input_ids* tensor.

In [6]:
inputs['labels'] = inputs.input_ids.detach().clone()

Now we mask tokens in the *input_ids* tensor, using the 15% probability we used before - and the **not** a *CLS*, *SEP*, or *PAD* token condition.

In [7]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

And now we take take the indices of each `True` value, within each individual vector.

In [8]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

Then apply these indices to each respective row in *input_ids*, assigning each of the values at these indices as *103*.

In [9]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

The `Trainer` expects a `Dataset` object, so we need to initialize this.

In [10]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        # return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize our data using the `MeditationsDataset` class.

In [11]:
dataset = MeditationsDataset(inputs)

We'll pass a training `args` dictionary to the `Trainer` defining our training arguments.

In [12]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=2,
    logging_dir='logs',
    optim="adamw_torch"
)

Now we'll import and initialize our `Trainer`.

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

And train.

In [14]:
trainer.train()

***** Running training *****
  Num examples = 507
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 64


RuntimeError: CUDA error: unknown error

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
import torch
import torch.nn as nn

torch.backends.cudnn.enabled=False
x = Variable(t.rand(1000000,1).contiguous()).cuda()

print(torch.backends.cudnn.version())

bn = nn.BatchNorm1d(1)
bn.cuda()

xbn = bn(x)
xbn.size()